# 1. Point Cloud Filtering

Computed or gathered pcds can sometimes be noisy or contain outliers. To reduce noise filtering techniques are used. Some filters are also used to reduce the point cloud density and thus reduce the computational cost.

### 1.1 Pass Through Filter

Applies constraints on the input data which are usually intervals along one or more axes. Points outside the specified range are removed.

In [4]:
import open3d as o3d
import math
import numpy as np
import itertools

In [22]:
pcd = o3d.io.read_point_cloud("./data/depth_2_pcd.ply")

points = np.asarray(pcd.points)

z_min = 0.8
z_max = 3.0

mask = (points[:,2] >= z_min) & (points[:,2] <= z_max)
filtered_points = points[mask]

filtered_pcd = o3d.geometry.PointCloud()
filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points)

# Visualize
# o3d.visualization.draw_geometries([filtered_pcd])

In [ ]:
# Another method for cropping/filtering
bounds = [[-math.inf, math.inf], [-math.inf, math.inf], [0.8, 3.0]]  # set the bounds
bounding_box_points = list(itertools.product(*bounds))  # create limit points
bounding_box = o3d.geometry.AxisAlignedBoundingBox.create_from_points(
    o3d.utility.Vector3dVector(bounding_box_points))  # create bounding box object

# Crop the point cloud using the bounding box:
pcd_cropped = pcd.crop(bounding_box)

# Display the cropped point cloud:
o3d.visualization.draw_geometries([pcd_cropped])

### 1.2 Down-sampling

o3d provides three different approaches to down-sample a point cloud:

1. **random_down_sample**: Randomly samples n * `sampling_ratio` points.
2. **uniform_down_sample**:  It selects a point every `every_k_points` points
3. **voxel_down_sample**: It creates a voxel grid and selects the centroid point per voxel. The size of the voxel is specified by the user. Time consuming.

In [23]:
# Random down-sampling:
random_pcd = pcd.random_down_sample(sampling_ratio=0.005)

# Uniform down-sampling:
uniform_pcd = pcd.uniform_down_sample(every_k_points=200)

# Voxel down-sampling:
voxel_pcd = pcd.voxel_down_sample(voxel_size=0.4)

# Translating point clouds:
points = np.asarray(random_pcd.points)
points += [-3, 3, 0]
random_pcd.points = o3d.utility.Vector3dVector(points)

points = np.asarray(uniform_pcd.points)
points += [0, 3, 0]
uniform_pcd.points = o3d.utility.Vector3dVector(points)

points = np.asarray(voxel_pcd.points)
points += [3, 3, 0]
voxel_pcd.points = o3d.utility.Vector3dVector(points)

# Display:
o3d.visualization.draw_geometries([pcd, random_pcd, uniform_pcd, voxel_pcd])

Note that the uniform pcd is uniformly distributed in the 3D space because the input pcd is an organized point cloud

In [24]:
# Shuffling to convert to an unorganized pcd
points = np.asarray(pcd.points)
np.random.shuffle(points)
u_pcd= o3d.geometry.PointCloud()
u_pcd.points= o3d.utility.Vector3dVector(points)

In [25]:
# performing downsampling on unorganized point cloud

# Random down-sampling:
random_u_pcd = u_pcd.random_down_sample(sampling_ratio=0.005)

# Uniform down-sampling:
uniform_u_pcd = u_pcd.uniform_down_sample(every_k_points=200)

# Voxel down-sampling:
voxel_u_pcd = u_pcd.voxel_down_sample(voxel_size=0.4)

# Translating point clouds:
points = np.asarray(random_u_pcd.points)
points += [-3, 3, 0]
random_u_pcd.points = o3d.utility.Vector3dVector(points)

points = np.asarray(uniform_u_pcd.points)
points += [0, 3, 0]
uniform_u_pcd.points = o3d.utility.Vector3dVector(points)

points = np.asarray(voxel_u_pcd.points)
points += [3, 3, 0]
voxel_u_pcd.points = o3d.utility.Vector3dVector(points)

# Display:
o3d.visualization.draw_geometries([u_pcd, random_u_pcd, uniform_u_pcd, voxel_u_pcd])

You'll now notice uniform downsample and random downsample are very similar. The difference is that uniform downsample is deterministic, while random downsample is not. This means that if you run the same code twice, uniform downsample will always return the same result, while random downsample will return different results.

However, voxel downsample returns a similar point cloud since it reorganizes the pointss into a 3D grid.

### 1.3 Outlier removal filters

* **Radius outlier removal**: Filter that removes every point that has less than a given number of neighbours within a given radius. <br><br>
* **Statistical outlier removal**: Filter that removes points that are further away from their neighbours. For each point its mean distance from all its neighbours is computed. Then, if the mean distance of the point is outside an interval defined by the global distances mean and std then the point is an outlier

In [28]:
# Down sampling to reduce the running time:
pcd = pcd.voxel_down_sample(voxel_size=0.02)

# Radius outlier removal:
pcd_rad, ind_rad = pcd.remove_radius_outlier(nb_points=16, radius=0.05)
outlier_rad_pcd = pcd.select_by_index(ind_rad, invert=True)
outlier_rad_pcd.paint_uniform_color([1., 0., 1.])

# Statistical outlier removal:
pcd_stat, ind_stat = pcd.remove_statistical_outlier(nb_neighbors=20,
                                                std_ratio=2.0)
outlier_stat_pcd = pcd.select_by_index(ind_stat, invert=True)
outlier_stat_pcd.paint_uniform_color([0., 0., 1.])

# Translate to visualize:
points = np.asarray(pcd_stat.points)
points += [3, 0, 0]
pcd_stat.points = o3d.utility.Vector3dVector(points)

points = np.asarray(outlier_stat_pcd.points)
points += [3, 0, 0]
outlier_stat_pcd.points = o3d.utility.Vector3dVector(points)

# Display:
o3d.visualization.draw_geometries([pcd_stat, pcd_rad, outlier_stat_pcd, outlier_rad_pcd])